In [1]:
import sys
# Alternate between ../../ and ../../src due to bug
sys.path.append("../../")

import pandas as pd
from src.config import config, ProductionConfig
from src.service.exchange import ExchangeService
from cryptolib.enums import ExchangeType

prod_config = ProductionConfig()

exchange = ExchangeService().get_exchange(ExchangeType.BINANCE.value, config.BINANCE_API_KEY, config.BINANCE_API_SECRET)
prod = ExchangeService().get_exchange(ExchangeType.BINANCE.value, prod_config.BINANCE_API_KEY, prod_config.BINANCE_API_SECRET)

# Checking if the values are the same
print(exchange.get_last_price('ETHUSDT'))
print(prod.get_last_price('ETHUSDT'))
print('Sandbox:', exchange.sandbox)

ModuleNotFoundError: No module named 'cryptolib'

In [3]:
import pandas as pd
data = exchange.get_klines('BTCUSDT', '1m', 100)
print(data)

# convert to pandas dataframe
df = pd.DataFrame(data, columns=['time', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_asset_volume', 'number_of_trades', 'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore'])
df

[[1684153440000, '27364.51000000', '27364.52000000', '27359.62000000', '27359.62000000', '5.54073000', 1684153499999, '151615.81721440', 278, '1.63639000', '44777.79675680', '0'], [1684153500000, '27359.62000000', '27359.63000000', '27351.58000000', '27351.58000000', '16.84457000', 1684153559999, '460799.89249430', 336, '5.00519000', '136917.39555650', '0'], [1684153560000, '27351.59000000', '27353.10000000', '27348.37000000', '27353.09000000', '20.16088000', 1684153619999, '551397.34065210', 433, '4.33420000', '118539.70454520', '0'], [1684153620000, '27353.09000000', '27363.54000000', '27353.09000000', '27363.53000000', '16.15123000', 1684153679999, '441870.37017180', 425, '10.61598000', '290433.64691430', '0'], [1684153680000, '27363.54000000', '27369.58000000', '27360.98000000', '27360.99000000', '27.36441000', 1684153739999, '748868.27732770', 469, '6.10046000', '166946.59048410', '0'], [1684153740000, '27360.98000000', '27360.99000000', '27347.01000000', '27347.02000000', '15.473

,time,open,high,low,close,volume,close_time,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,ignore
0,1684153440000,27364.51000000,27364.52000000,27359.62000000,27359.62000000,5.54073000,1684153499999,151615.81721440,278,1.63639000,44777.79675680,0
1,1684153500000,27359.62000000,27359.63000000,27351.58000000,27351.58000000,16.84457000,1684153559999,460799.89249430,336,5.00519000,136917.39555650,0
2,1684153560000,27351.59000000,27353.10000000,27348.37000000,27353.09000000,20.16088000,1684153619999,551397.34065210,433,4.33420000,118539.70454520,0
3,1684153620000,27353.09000000,27363.54000000,27353.09000000,27363.53000000,16.15123000,1684153679999,441870.37017180,425,10.61598000,290433.64691430,0
4,1684153680000,27363.54000000,27369.58000000,27360.98000000,27360.99000000,27.36441000,1684153739999,748868.27732770,469,6.10046000,166946.59048410,0
...,...,...,...,...,...,...,...,...,...,...,...,...
95,1684159140000,27369.96000000,27374.25000000,27366.16000000,27374.25000000,10.04635000,1684159199999,274967.00656220,348,2.78401000,76198.62649320,0
96,1684159200000,27374.25000000,27374.25000000,27368.19000000,27369.45000000,13.27608000,1684159259999,363390.13433020,490,6.51795000,178399.45561110,0
97,1684159260000,27369.44000000,27376.42000000,27369.44000000,27370.58000000,15.45219000,1684159319999,422987.55649530,483,9.77527000,267585.51552050,0
98,1684159320000,27370.58000000,27370.59000000,27361.86000000,27361.98000000,19.70198000,1684159379999,539171.60498480,484,4.08347000,111736.29021770,0
